# Mobile price classification with multilayer perceptron
## Tensorflow
Erik Matovič and Jakub Horvat

[Dataset](https://www.kaggle.com/datasets/iabhishekofficial/mobile-price-classification?select=train.csv)

Predict a price range indicating how high the price is

### 0. Imports

In [227]:
from utils import split_train_val
import tensorflow as tf
from tensorflow import keras
import pandas as pd
#import wandb
from utils import check_null_values, print_sum_null, rescale, split_data

In [228]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.reduce_sum(tf.random.normal([1000, 1000])))
print(tf.config.list_physical_devices('GPU'))


2.11.0
Num GPUs Available:  0
tf.Tensor(-123.019745, shape=(), dtype=float32)
[]


### 1. Data Preprocessing

Based on [exploratory data analysis](EDA.ipynb) test set does not have target variable price_range. We split our dataset into train-dev-test. We have train and test sets, but we split test set by half to dev-test sets. We will rougly have train-dev-test 67%-16.5%-16.5%. 

In [229]:
# read datasets
df_train = pd.read_csv('../data/train.csv', sep=',')
df_train


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [230]:
df_test = pd.read_csv('../data/test.csv', sep=',', index_col=0)
df_test


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
id,,,,,,,,,,,,,,,,,,,,
1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0
2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0
3,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1
4,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0
5,1434,0,1.4,0,11,1,49,0.5,108,6,18,749,810,1773,15,8,7,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1700,1,1.9,0,0,1,54,0.5,170,7,17,644,913,2121,14,8,15,1,1,0
997,609,0,1.8,1,0,0,13,0.9,186,4,2,1152,1632,1933,8,1,19,0,1,1
998,1185,0,1.4,0,1,1,8,0.5,80,1,12,477,825,1223,5,0,14,1,0,0


In [231]:
df_train.values

array([[8.420e+02, 0.000e+00, 2.200e+00, ..., 0.000e+00, 1.000e+00,
        1.000e+00],
       [1.021e+03, 1.000e+00, 5.000e-01, ..., 1.000e+00, 0.000e+00,
        2.000e+00],
       [5.630e+02, 1.000e+00, 5.000e-01, ..., 1.000e+00, 0.000e+00,
        2.000e+00],
       ...,
       [1.911e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 0.000e+00,
        3.000e+00],
       [1.512e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [5.100e+02, 1.000e+00, 2.000e+00, ..., 1.000e+00, 1.000e+00,
        3.000e+00]])

In [232]:
x_train = df_train.values[:, :-1]       #vsetko okrem price range
x_train = x_train.astype('float32')     # input data as floats
x_train


array([[8.420e+02, 0.000e+00, 2.200e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [1.021e+03, 1.000e+00, 5.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [5.630e+02, 1.000e+00, 5.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [1.911e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [1.512e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [5.100e+02, 1.000e+00, 2.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00]], dtype=float32)

In [233]:
y_train = df_train.values[:, -1]  # price range
y_train = y_train.astype('int')
print(y_train)

[1 2 2 ... 3 0 3]


In [234]:
x_test = df_test.values[:, :-1]       #vsetko okrem price range
x_test = x_test.astype('float32')

y_test = df_test.values[:, -1]  # price range
y_test = y_test.astype('int')
print(x_test.shape,y_test.shape)

(1000, 19) (1000,)


In [235]:
# train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#
# # Print the dataset
# for data, label in train_ds:
#     print(data, label)

In [236]:
# batch_size = 32
# tf_train, tf_val = split_train_val(train_ds)
#
# print(tf_train.dataset)
# print(tf_val.dataset)


In [237]:
#shape(tf_train)

In [238]:
model = keras.Sequential([
    keras.layers.Dense(16,input_shape=(20,),name='l1'),
    keras.layers.Dense(16, activation='relu',name='l2'),
    keras.layers.Dense(16, activation='relu',name='l3'),
    keras.layers.Dense(1,activation='sigmoid',name='l4')
])


In [239]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [240]:
model.fit(x_train,y_train,batch_size=32,epochs=100)


Epoch 1/100
63/63 [==============================] - 2s 2ms/step - loss: -613.6506 - accuracy: 0.2615
Epoch 2/100
63/63 [==============================] - 0s 2ms/step - loss: -4578.7559 - accuracy: 0.2500
Epoch 3/100
63/63 [==============================] - 0s 2ms/step - loss: -24462.4434 - accuracy: 0.2500
Epoch 4/100
63/63 [==============================] - 0s 2ms/step - loss: -95599.8203 - accuracy: 0.2500
Epoch 5/100
63/63 [==============================] - 0s 2ms/step - loss: -285463.5938 - accuracy: 0.2500
Epoch 6/100
63/63 [==============================] - 0s 2ms/step - loss: -695407.3750 - accuracy: 0.2500
Epoch 7/100
63/63 [==============================] - 0s 2ms/step - loss: -1442993.1250 - accuracy: 0.2500
Epoch 8/100
63/63 [==============================] - 0s 2ms/step - loss: -2658539.7500 - accuracy: 0.2500
Epoch 9/100
63/63 [==============================] - 0s 2ms/step - loss: -4496711.5000 - accuracy: 0.2500
Epoch 10/100
63/63 [==============================] - 0s 2m

In [241]:
# results = model.evaluate(x_test, y_test, verbose = 0)
# print('test loss, test acc:', results)

ValueError: in user code:

    File "D:\skola\ING\kpais\cv4\weeklyTasks\mobile-price-classification\lib\site-packages\keras\engine\training.py", line 1820, in test_function  *
        return step_function(self, iterator)
    File "D:\skola\ING\kpais\cv4\weeklyTasks\mobile-price-classification\lib\site-packages\keras\engine\training.py", line 1804, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\skola\ING\kpais\cv4\weeklyTasks\mobile-price-classification\lib\site-packages\keras\engine\training.py", line 1792, in run_step  **
        outputs = model.test_step(data)
    File "D:\skola\ING\kpais\cv4\weeklyTasks\mobile-price-classification\lib\site-packages\keras\engine\training.py", line 1756, in test_step
        y_pred = self(x, training=False)
    File "D:\skola\ING\kpais\cv4\weeklyTasks\mobile-price-classification\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered